In [ ]:
!pip install BeautifulSoup4

Looking in indexes: http://ftp.daumkakao.com/pypi/simple
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1279 sha256=6c4637397512d828506866c56ff01da800517cce964d3075634e46dcc1655e2f
  Stored in directory: c:\users\이소정\appdata\local\pip\cache\wheels\53\1c\ea\8f4f03db1693f3bdb924afce36bc9296aff50d512553fccb10
Successfully built bs4
Looking in indexes: http://ftp.daumkakao.com/pypi/simple


In [ ]:
import requests as rq
import pandas as pd
from bs4 import BeautifulSoup as bs

In [ ]:
import xml.etree.ElementTree as ET
from xml.etree.ElementTree import Element, dump, ElementTree

In [ ]:
import pymysql as pms
import datetime

### 1. 판례 목록 나와있는 url 가져오기
> 입력 변수
* OC	string(필수)	사용자 이메일의 ID(g4c@korea.kr일경우 OC값=g4c)
* target	string : prec(필수)	서비스 대상
* type	char(필수)	출력 형태 : HTML / XML
* search	int	검색범위 (기본 : 1 판례명) 2 : 본문검색
* query	string	검색범위에서 검색을 원하는 질의(검색 결과 리스트)
* display	int	검색된 결과 개수 (default=20 max=100)
* page	int	검색 결과 페이지 (default=1)
* org	string	법원종류 (대법원:400201, 하위법원:400202)
* curt	string	법원명 (대법원, 서울고등법원, 광주지법, 인천지방법원)
* JO	string	참조법령명(형법, 민법 등)
* gana	string	사전식 검색(ga,na,da…,etc)
* sort	string	정렬옵션
* lasc : 사건명 오름차순
* ldes : 사건명 내림차순
* dasc : 선고일자 오름차순
* ddes : 선고일자 내림차순(생략시 기본)
* nasc : 법원명 오름차순
* ndes : 법원명 내림차순
* date	int	판례 선고일자
* prncYd	string	선고일자 검색(20090101~20090130)
* nb	int	판례 사건번호
* popYn	string	상세화면 팝업창 여부(팝업창으로 띄우고 싶을 때만 'popYn=Y')
> ##### (필수) 제외하면 안들어가도 되는 변수임
</br>

* #### 목록 parameter 딕셔너리


> query가 keyword에 해당하는데, input 함수로 키워드를 입력받음
<br/> 여기선 키워드로 손해배상 입력, 보여줄 판례의 수는 10개, 법원은 대법원, 기간은 20100101~20191231로 설정함
<br/> type은 url 타고 웹에서 볼거면 html로 하는게 낫고, 주피터노트북으로 text 확인할거면 xml이 나음

In [ ]:
chart= {'OC':'ICTPoolC'
        ,'target':'prec'
        ,'type':'XML'
        ,'query':input()
        ,'display':10
        ,'curt':'대법원'
        ,'prncYd':'20100101~20191231'
       }

손해배상


* #### request 모듈을 이용하여 chart의 변수들을 전송함

In [ ]:
res_chart=rq.get('http://www.law.go.kr/DRF/lawSearch.do?',params=chart)

* #### url 가져오기

In [ ]:
print(res_chart.url)

http://www.law.go.kr/DRF/lawSearch.do?OC=ICTPoolC&target=prec&type=XML&query=%EC%86%90%ED%95%B4%EB%B0%B0%EC%83%81&display=10&curt=%EB%8C%80%EB%B2%95%EC%9B%90&prncYd=20100101~20191231


* #### text 가져오기
> 아래에서 볼 수 있듯이 html 타입으로 text 읽어오는건 무의미한듯..

In [ ]:
print(res_chart.text)

<?xml version="1.0" encoding="UTF-8"?><PrecSearch><target>prec</target><키워드>손해배상</키워드><section>evtNm</section><totalCnt>874</totalCnt><page>1</page><prec id="1"><판례일련번호>210569</판례일련번호><사건명><![CDATA[<strong class="tbl_tx_type"><strong class="tbl_tx_type">손해</strong><strong class="tbl_tx_type">배상</strong></strong>(기)]]></사건명><사건번호>2018다233686</사건번호><선고일자>2019.11.14</선고일자><법원명>대법원</법원명><법원종류코드></법원종류코드><사건종류명>민사</사건종류명><사건종류코드>400101</사건종류코드><판결유형>판결</판결유형><선고>선고</선고><판례상세링크>/DRF/lawService.do?OC=ICTPoolC&amp;target=prec&amp;ID=210569&amp;type=HTML&amp;mobileYn=</판례상세링크></prec><prec id="2"><판례일련번호>210415</판례일련번호><사건명><![CDATA[<strong class="tbl_tx_type"><strong class="tbl_tx_type">손해</strong><strong class="tbl_tx_type">배상</strong></strong>(기)]]></사건명><사건번호>2018다233242</사건번호><선고일자>2019.10.31</선고일자><법원명>대법원</법원명><법원종류코드></법원종류코드><사건종류명>민사</사건종류명><사건종류코드>400101</사건종류코드><판결유형>판결</판결유형><선고>선고</선고><판례상세링크>/DRF/lawService.do?OC=ICTPoolC&amp;target=prec&amp;ID=210415&amp;type=HTML&amp;mobileYn=</

### 2. 판례 본문 url 가져오기
> 입력 변수
* OC	string(필수)	사용자 이메일의 ID(g4c@korea.kr일경우 OC값=g4c)
* target	string : prec(필수)	서비스 대상
* type	char(필수)	출력 형태 : HTML / XML
* ID	char(필수)	판례 일련번호
* LM	string	판례명
> ##### (필수) 제외하면 안들어가도 되는 변수임
</br>

* #### 본문 parameter 딕셔너리


> ID가 판례일련번호에 해당하는데, input 함수로 판례일련번호를 입력받음

In [ ]:
main = {'OC':'ICTPoolC',
           'target':'prec',
           'ID': int(input()),
          'type':'XML'}

210569


* #### request 모듈을 이용하여 main의 변수들을 전송함

In [ ]:
res_main= rq.get('http://www.law.go.kr/DRF/lawService.do?',params=main)

* #### url 가져오기

In [ ]:
print(res_main.url)

http://www.law.go.kr/DRF/lawService.do?OC=ICTPoolC&target=prec&ID=210569&type=XML


### 3. 판례 목록에서 판례일련번호만 리스트에 저장하기
> tag로부터 판례일련번호를 가져와야하므로 beautifulSoup 모듈이용

In [ ]:
!pip uninstall lxml

In [ ]:
soup = bs(res_chart.text,'xml')

In [ ]:
find = soup.find_all('판례일련번호')

In [ ]:
find

[<판례일련번호>210569</판례일련번호>,
 <판례일련번호>210415</판례일련번호>,
 <판례일련번호>210423</판례일련번호>,
 <판례일련번호>210419</판례일련번호>,
 <판례일련번호>209687</판례일련번호>,
 <판례일련번호>209693</판례일련번호>,
 <판례일련번호>209587</판례일련번호>,
 <판례일련번호>209131</판례일련번호>,
 <판례일련번호>208800</판례일련번호>,
 <판례일련번호>208795</판례일련번호>]

#### 번호만 가져와야 하므로 slicing 이용

In [ ]:
num_list = []
for k in find:
    line = str(k)
    num_list.append(int(line[8:14]))

In [ ]:
num_list

[210569,
 210415,
 210423,
 210419,
 209687,
 209693,
 209587,
 209131,
 208800,
 208795]

### 4. 함수
> 두가지의 함수를 만들었다
> * 키워드와 열람하고싶은 판례의 수를 입력하면, 해당 판례들의 일련번호를 리스트로 반환하는 함수
> * 위의 함수를 이용해 일련번호 리스트 안의 각 번호에 해당하는 판례의 url을 제공하는 함수

In [ ]:
def get_numbers(keyword,num):
    chart= {'OC':'ICTPoolC'
        ,'target':'prec'
        ,'type':'XML'
        ,'query':keyword
        ,'display':num
        ,'curt':'대법원'
        ,'prncYd':'20100101~20191231'
       }
    res_chart=rq.get('http://www.law.go.kr/DRF/lawSearch.do?',params=chart)
    soup = bs(res_chart.text,'xml')
    find = soup.find_all('판례일련번호')
    num_list = []
    for k in find:
        line = str(k)
        num_list.append(int(line[8:14]))
    return num_list

In [ ]:
get_numbers('손해배상',10)

[210569,
 210415,
 210423,
 210419,
 209687,
 209693,
 209587,
 209131,
 208800,
 208795]

In [ ]:
def get_url(keyword,num):
    num_list = get_numbers(keyword,num)
    url_list = []
    for i in num_list:
        main = {'OC':'ICTPoolC',
           'target':'prec',
           'ID': i,
          'type':'HTML'}
        res_main= rq.get('http://www.law.go.kr/DRF/lawService.do?',params=main)
        url_list.append(res_main.url)
    for url in url_list:
        print(url)

In [ ]:
get_url('손해배상', 10)

http://www.law.go.kr/DRF/lawService.do?OC=ICTPoolC&target=prec&ID=210569&type=HTML
http://www.law.go.kr/DRF/lawService.do?OC=ICTPoolC&target=prec&ID=210415&type=HTML
http://www.law.go.kr/DRF/lawService.do?OC=ICTPoolC&target=prec&ID=210423&type=HTML
http://www.law.go.kr/DRF/lawService.do?OC=ICTPoolC&target=prec&ID=210419&type=HTML
http://www.law.go.kr/DRF/lawService.do?OC=ICTPoolC&target=prec&ID=209687&type=HTML
http://www.law.go.kr/DRF/lawService.do?OC=ICTPoolC&target=prec&ID=209693&type=HTML
http://www.law.go.kr/DRF/lawService.do?OC=ICTPoolC&target=prec&ID=209587&type=HTML
http://www.law.go.kr/DRF/lawService.do?OC=ICTPoolC&target=prec&ID=209131&type=HTML
http://www.law.go.kr/DRF/lawService.do?OC=ICTPoolC&target=prec&ID=208800&type=HTML
http://www.law.go.kr/DRF/lawService.do?OC=ICTPoolC&target=prec&ID=208795&type=HTML


In [ ]:
### 4. DB연동

In [ ]:
!pip install cryptography

Looking in indexes: http://ftp.daumkakao.com/pypi/simple


In [ ]:
# 1. DB 연결
conn = pms.connect(host="192.168.0.15", port=3306, user="ict", password="48834883", db="ict", charset="utf8")

In [ ]:
soup = bs(res_chart.text,'xml')

In [ ]:
ids = soup.find_all('판례일련번호')
ids

[<판례일련번호>210569</판례일련번호>,
 <판례일련번호>210415</판례일련번호>,
 <판례일련번호>210423</판례일련번호>,
 <판례일련번호>210419</판례일련번호>,
 <판례일련번호>209687</판례일련번호>,
 <판례일련번호>209693</판례일련번호>,
 <판례일련번호>209587</판례일련번호>,
 <판례일련번호>209131</판례일련번호>,
 <판례일련번호>208800</판례일련번호>,
 <판례일련번호>208795</판례일련번호>]

In [ ]:
id_list = []
for id in ids:
    line = str(id)
    id_list.append(int(line[8:-9]))
id_list

[210569,
 210415,
 210423,
 210419,
 209687,
 209693,
 209587,
 209131,
 208800,
 208795]

In [ ]:
caseNames = soup.find_all('사건명')
caseNames

[<사건명>&lt;strong class="tbl_tx_type"&gt;&lt;strong class="tbl_tx_type"&gt;손해&lt;/strong&gt;&lt;strong class="tbl_tx_type"&gt;배상&lt;/strong&gt;&lt;/strong&gt;(기)</사건명>,
 <사건명>&lt;strong class="tbl_tx_type"&gt;&lt;strong class="tbl_tx_type"&gt;손해&lt;/strong&gt;&lt;strong class="tbl_tx_type"&gt;배상&lt;/strong&gt;&lt;/strong&gt;(기)</사건명>,
 <사건명>&lt;strong class="tbl_tx_type"&gt;&lt;strong class="tbl_tx_type"&gt;손해&lt;/strong&gt;&lt;strong class="tbl_tx_type"&gt;배상&lt;/strong&gt;&lt;/strong&gt;(기)</사건명>,
 <사건명>&lt;strong class="tbl_tx_type"&gt;&lt;strong class="tbl_tx_type"&gt;손해&lt;/strong&gt;&lt;strong class="tbl_tx_type"&gt;배상&lt;/strong&gt;&lt;/strong&gt;(기)</사건명>,
 <사건명>&lt;strong class="tbl_tx_type"&gt;&lt;strong class="tbl_tx_type"&gt;손해&lt;/strong&gt;&lt;strong class="tbl_tx_type"&gt;배상&lt;/strong&gt;&lt;/strong&gt;(기)·&lt;strong class="tbl_tx_type"&gt;&lt;strong class="tbl_tx_type"&gt;손해&lt;/strong&gt;&lt;strong class="tbl_tx_type"&gt;배상&lt;/strong&gt;&lt;/strong&gt;(기)·&lt;strong c

In [ ]:
caseName_list = []
for title in caseNames:
    line = title.string
    line = line.replace("<strong class=\"tbl_tx_type\">", "")
    line = line.replace("</strong>", "")
    caseName_list.append(line)
caseName_list

['손해배상(기)',
 '손해배상(기)',
 '손해배상(기)',
 '손해배상(기)',
 '손해배상(기)·손해배상(기)·손해배상(기)',
 '손해배상(자)(교통사고 피해자의 유족들이 가해자와 보험사를 상대로 손해배상을 구하는 사건)',
 '영업비밀침해금지및손해배상등',
 '손해배상',
 '손해배상(기)',
 '손해배상(자)(교통사고로 발생한 장애로 인한 손해배상청구권의 소멸시효의 기산일)']

In [ ]:
caseNums = soup.find_all('사건번호')
caseNums

[<사건번호>2018다233686</사건번호>,
 <사건번호>2018다233242</사건번호>,
 <사건번호>2017다282438</사건번호>,
 <사건번호>2016다243306</사건번호>,
 <사건번호>2018다222303</사건번호>,
 <사건번호>2017다280951</사건번호>,
 <사건번호>2017다34981</사건번호>,
 <사건번호>2016다217833</사건번호>,
 <사건번호>2019다206933</사건번호>,
 <사건번호>2016다1687</사건번호>]

In [ ]:
caseNum_list = []
for num in caseNums:
    line = str(num)
    caseNum_list.append(line[6:-7])
caseNum_list

['2018다233686',
 '2018다233242',
 '2017다282438',
 '2016다243306',
 '2018다222303',
 '2017다280951',
 '2017다34981',
 '2016다217833',
 '2019다206933',
 '2016다1687']

In [ ]:
dates = soup.find_all('선고일자')
dates

[<선고일자>2019.11.14</선고일자>,
 <선고일자>2019.10.31</선고일자>,
 <선고일자>2019.10.31</선고일자>,
 <선고일자>2019.10.31</선고일자>,
 <선고일자>2019.09.26</선고일자>,
 <선고일자>2019.09.26</선고일자>,
 <선고일자>2019.09.10</선고일자>,
 <선고일자>2019.08.14</선고일자>,
 <선고일자>2019.07.25</선고일자>,
 <선고일자>2019.07.25</선고일자>]

In [ ]:
date_list = []
for date in dates:
    line = str(date)
    date_list.append(datetime.date(int(line[6:10]), int(line[11:13]), int(line[14:16])))
date_list

[datetime.date(2019, 11, 14),
 datetime.date(2019, 10, 31),
 datetime.date(2019, 10, 31),
 datetime.date(2019, 10, 31),
 datetime.date(2019, 9, 26),
 datetime.date(2019, 9, 26),
 datetime.date(2019, 9, 10),
 datetime.date(2019, 8, 14),
 datetime.date(2019, 7, 25),
 datetime.date(2019, 7, 25)]

In [ ]:
courtNames = soup.find_all('법원명')
courtNames

[<법원명>대법원</법원명>,
 <법원명>대법원</법원명>,
 <법원명>대법원</법원명>,
 <법원명>대법원</법원명>,
 <법원명>대법원</법원명>,
 <법원명>대법원</법원명>,
 <법원명>대법원</법원명>,
 <법원명>대법원</법원명>,
 <법원명>대법원</법원명>,
 <법원명>대법원</법원명>]

In [ ]:
courtName_list = []
for courtName in courtNames:
    line = str(courtName)
    courtName_list.append(line[5:-6])
courtName_list

['대법원', '대법원', '대법원', '대법원', '대법원', '대법원', '대법원', '대법원', '대법원', '대법원']

In [ ]:
courtIDs = soup.find_all('법원종류코드')
courtIDs

[<법원종류코드/>,
 <법원종류코드/>,
 <법원종류코드/>,
 <법원종류코드/>,
 <법원종류코드/>,
 <법원종류코드/>,
 <법원종류코드/>,
 <법원종류코드/>,
 <법원종류코드/>,
 <법원종류코드/>]

In [ ]:
courtID_list = []
for courtID in courtIDs:
    line = str(courtID)
    if line[8:-9] == "":
        courtID_list.append("NULL")
    else:
        courtID_list.append(int(line[8:-9]))
courtID_list

['NULL',
 'NULL',
 'NULL',
 'NULL',
 'NULL',
 'NULL',
 'NULL',
 'NULL',
 'NULL',
 'NULL']

In [ ]:
caseTypes = soup.find_all('사건종류명')
caseTypes

[<사건종류명>민사</사건종류명>,
 <사건종류명>민사</사건종류명>,
 <사건종류명>민사</사건종류명>,
 <사건종류명>민사</사건종류명>,
 <사건종류명>민사</사건종류명>,
 <사건종류명>민사</사건종류명>,
 <사건종류명>민사</사건종류명>,
 <사건종류명>민사</사건종류명>,
 <사건종류명>민사</사건종류명>,
 <사건종류명>민사</사건종류명>]

In [ ]:
caseType_list = []
for caseType in caseTypes:
    line = str(caseType)
    caseType_list.append(line[7:-8])
caseType_list

['민사', '민사', '민사', '민사', '민사', '민사', '민사', '민사', '민사', '민사']

In [ ]:
caseTypeIDs = soup.find_all('사건종류코드')
caseTypeIDs

[<사건종류코드>400101</사건종류코드>,
 <사건종류코드>400101</사건종류코드>,
 <사건종류코드>400101</사건종류코드>,
 <사건종류코드>400101</사건종류코드>,
 <사건종류코드>400101</사건종류코드>,
 <사건종류코드>400101</사건종류코드>,
 <사건종류코드>400101</사건종류코드>,
 <사건종류코드>400101</사건종류코드>,
 <사건종류코드>400101</사건종류코드>,
 <사건종류코드>400101</사건종류코드>]

In [ ]:
caseTypeID_list = []
for caseTypeID in caseTypeIDs:
    line = str(caseTypeID)
    if line[8:-9] == "":
        caseTypeID_list.append("NULL")
    else:
        caseTypeID_list.append(int(line[8:-9]))
caseTypeID_list

[400101,
 400101,
 400101,
 400101,
 400101,
 400101,
 400101,
 400101,
 400101,
 400101]

In [ ]:
verdictTypes = soup.find_all('판결유형')
verdictTypes

[<판결유형>판결</판결유형>,
 <판결유형>판결</판결유형>,
 <판결유형>판결</판결유형>,
 <판결유형>판결</판결유형>,
 <판결유형>판결</판결유형>,
 <판결유형>판결</판결유형>,
 <판결유형>판결</판결유형>,
 <판결유형>판결</판결유형>,
 <판결유형>판결</판결유형>,
 <판결유형>판결</판결유형>]

In [ ]:
verdictType_list = []
for verdictType in verdictTypes:
    line = str(verdictType)
    if line[6:-7] == "":
        verdictType_list.append("NULL")
    else:
        verdictType_list.append(line[6:-7])
verdictType_list

['판결', '판결', '판결', '판결', '판결', '판결', '판결', '판결', '판결', '판결']

In [ ]:
sentences = soup.find_all('선고')
sentences

[<선고>선고</선고>,
 <선고>선고</선고>,
 <선고>선고</선고>,
 <선고>선고</선고>,
 <선고>선고</선고>,
 <선고>선고</선고>,
 <선고>선고</선고>,
 <선고>선고</선고>,
 <선고>선고</선고>,
 <선고>선고</선고>]

In [ ]:
sentence_list = []
for sentence in sentences:
    line = str(sentence)
    if line[4:-5] == "":
        sentence_list.append("NULL")
    else:
        sentence_list.append(line[4:-5])
sentence_list

['선고', '선고', '선고', '선고', '선고', '선고', '선고', '선고', '선고', '선고']

In [ ]:
exURLs = soup.find_all('판례상세링크')
exURLs

[<판례상세링크>/DRF/lawService.do?OC=ICTPoolC&amp;target=prec&amp;ID=210569&amp;type=HTML&amp;mobileYn=</판례상세링크>,
 <판례상세링크>/DRF/lawService.do?OC=ICTPoolC&amp;target=prec&amp;ID=210415&amp;type=HTML&amp;mobileYn=</판례상세링크>,
 <판례상세링크>/DRF/lawService.do?OC=ICTPoolC&amp;target=prec&amp;ID=210423&amp;type=HTML&amp;mobileYn=</판례상세링크>,
 <판례상세링크>/DRF/lawService.do?OC=ICTPoolC&amp;target=prec&amp;ID=210419&amp;type=HTML&amp;mobileYn=</판례상세링크>,
 <판례상세링크>/DRF/lawService.do?OC=ICTPoolC&amp;target=prec&amp;ID=209687&amp;type=HTML&amp;mobileYn=</판례상세링크>,
 <판례상세링크>/DRF/lawService.do?OC=ICTPoolC&amp;target=prec&amp;ID=209693&amp;type=HTML&amp;mobileYn=</판례상세링크>,
 <판례상세링크>/DRF/lawService.do?OC=ICTPoolC&amp;target=prec&amp;ID=209587&amp;type=HTML&amp;mobileYn=</판례상세링크>,
 <판례상세링크>/DRF/lawService.do?OC=ICTPoolC&amp;target=prec&amp;ID=209131&amp;type=HTML&amp;mobileYn=</판례상세링크>,
 <판례상세링크>/DRF/lawService.do?OC=ICTPoolC&amp;target=prec&amp;ID=208800&amp;type=HTML&amp;mobileYn=</판례상세링크>,
 <판례상세링크>/DRF/lawService.do?

In [ ]:
exURL_list = []
for exURL in exURLs:
    line = str(exURL)
    exURL_list.append(line[8:-9])
exURL_list

['/DRF/lawService.do?OC=ICTPoolC&amp;target=prec&amp;ID=210569&amp;type=HTML&amp;mobileYn=',
 '/DRF/lawService.do?OC=ICTPoolC&amp;target=prec&amp;ID=210415&amp;type=HTML&amp;mobileYn=',
 '/DRF/lawService.do?OC=ICTPoolC&amp;target=prec&amp;ID=210423&amp;type=HTML&amp;mobileYn=',
 '/DRF/lawService.do?OC=ICTPoolC&amp;target=prec&amp;ID=210419&amp;type=HTML&amp;mobileYn=',
 '/DRF/lawService.do?OC=ICTPoolC&amp;target=prec&amp;ID=209687&amp;type=HTML&amp;mobileYn=',
 '/DRF/lawService.do?OC=ICTPoolC&amp;target=prec&amp;ID=209693&amp;type=HTML&amp;mobileYn=',
 '/DRF/lawService.do?OC=ICTPoolC&amp;target=prec&amp;ID=209587&amp;type=HTML&amp;mobileYn=',
 '/DRF/lawService.do?OC=ICTPoolC&amp;target=prec&amp;ID=209131&amp;type=HTML&amp;mobileYn=',
 '/DRF/lawService.do?OC=ICTPoolC&amp;target=prec&amp;ID=208800&amp;type=HTML&amp;mobileYn=',
 '/DRF/lawService.do?OC=ICTPoolC&amp;target=prec&amp;ID=208795&amp;type=HTML&amp;mobileYn=']

In [ ]:
# 2. 커서 생성 (트럭, 연결로프)
cur = conn.cursor()

for i in range (0, len(id_list)):
    sql = "INSERT INTO exlist VALUES( %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
    val = (id_list[i], caseName_list[i], caseNum_list[i], date_list[i], courtName_list[i], int(), caseType_list[i], caseTypeID_list[i], verdictType_list[i], sentence_list[i], exURL_list[i])
    cur.execute(sql, val)

cur.close()
conn.commit()
conn.close() # 6. DB 닫기 (=연결 해제)
print('OK~')

OK~
